# 🚀 FoadsIA Backend Server v2.0 (Optimized for T4)

Este notebook despliega el servidor backend de **FoadsIA** optimizado para Google Colab (T4 GPU).

### ✨ Características:
- **SDXL Lightning**: Generación de imágenes en <4 segundos.
- **Face Swap**: Integración con InsightFace.
- **Video Rendering**: Avatares parlantes con Edge-TTS y SadTalker (lite).
- **Seguridad**: Autenticación JWT y Rate Limiting activados.
- **Fix Automático**: Resuelve problemas de carga de modelos .safetensors.

---

In [ ]:
#@title 1. Inicialización y Configuración del Entorno
#@markdown Clona el repositorio, instala dependencias del sistema y librerías de Python.

import os
import sys
from google.colab import drive, userdata

# 1. Verificar GPU
!nvidia-smi

# 2. Clonar/Actualizar Repositorio
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
REPO_DIR = "/content/FoadsIA"

if not os.path.exists(REPO_DIR):
    print("📦 Clonando repositorio FoadsIA...")
    !git clone {REPO_URL}
else:
    print("🔄 Actualizando repositorio...")
    %cd {REPO_DIR}
    !git pull

%cd {REPO_DIR}

# 3. Instalar Dependencias del Sistema
print("🛠️ Instalando FFmpeg y herramientas...")
!apt-get update -qq && apt-get install -y ffmpeg aria2 -qq

# 4. Instalar Dependencias de Python
print("🐍 Instalando librerías de Python (esto puede tardar unos minutos)...")
!pip install -r backend/requirements.txt

# Fixes específicos para Colab
!pip install --upgrade diffusers==0.26.3 transformers==4.38.2 accelerate==0.27.2 safetensors==0.4.2
!pip install pyngrok flask-socketio eventlet python-socketio onnxruntime-gpu

print("✅ Entorno preparado exitosamente.")

In [ ]:
#@title 2. Descarga de Modelos (Opcional - Se hace auto al iniciar)
#@markdown Ejecuta esto si quieres pre-descargar los modelos antes de iniciar el servidor.

%cd /content/FoadsIA
print("⬇️ Verificando y descargando modelos...")
!python backend/master_loader.py

In [ ]:
#@title 3. Iniciar Servidor (Ngrok + Flask)
#@markdown Inicia el servidor y genera la URL pública.

from pyngrok import ngrok
import getpass
import os

# Configuración de Ngrok
ngrok.kill()

# Intentar obtener token de secretos de Colab, sino pedirlo
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
except:
    NGROK_TOKEN = None

if not NGROK_TOKEN:
    print("🔑 Introduce tu Ngrok Authtoken (consíguelo gratis en dashboard.ngrok.com):")
    NGROK_TOKEN = getpass.getpass('Enter Ngrok Authtoken: ')

if NGROK_TOKEN:
    !ngrok config add-authtoken {NGROK_TOKEN}
    
    # Crear túnel
    public_url = ngrok.connect(5000).public_url
    print("\n" + "="*60)
    print(f"🚀 URL PÚBLICA DEL BACKEND: \033[92m{public_url}\033[0m")
    print("="*60 + "\n")
    print("👉 Copia esta URL y pégala en: Settings -> API URL en tu Frontend\n")
    
    # Configurar Variables de Entorno
    os.environ['BASE_URL'] = public_url
    os.environ['JWT_SECRET'] = "colab-dev-secret-key-2025" # Cambiar claves en producción
    
    # Iniciar Servidor Flask
    %cd /content/FoadsIA
    print("🔥 Iniciando Servidor Flask...")
    !python backend/app.py
    
else:
    print("❌ Error: Se requiere un token de Ngrok para continuar.")